In [43]:
import os, json
import jsonlines

DATA_DIR = "../data"
data_list = [
    "KoAlpaca/KoAlpaca_v1.1.jsonl",
    "KoChatGPT/kochatgpt_1_SFT.jsonl",
    "korquad-chat-v1/korquad-chat.json",
    "OIG-small-chip2-ko/oig-smallchip2-dedu.jsonl",
    "ShareGPT_DeepL/ko_alpaca_style_dataset.json",
    "kullm_v2/kullm-v2.jsonl",
    "KoAlpaca/ko_alpaca_data.json"]

### KoAlpaca v1.0 dataset

In [44]:
data_path = data_list[-1]

with open(os.path.join(DATA_DIR, data_path), "r") as f:
    koalpaca_v1_0 = json.load(f)

len(koalpaca_v1_0)

49620

### KoAlpaca v1.1 dataset

In [45]:
data_path = data_list[0]

koalpaca_v1_1 = []
with jsonlines.open(os.path.join(DATA_DIR, data_path), "r") as f:
    for line in f.iter():
        koalpaca_v1_1.append(line)

len(koalpaca_v1_1)

21155

### KochatGPT SFT dataset

In [46]:
data_path = data_list[1]
with open(os.path.join(DATA_DIR, data_path), "r") as f:
    kochatgpt = json.load(f)

len(kochatgpt)

12000

### Korquad dataset

In [47]:
data_path = data_list[2]

korquad = []
with jsonlines.open(os.path.join(DATA_DIR, data_path), "r") as f:
    for line in f.iter():
        korquad.append(line)

len(korquad)

9619

### OIG dataset

In [48]:
data_path = data_list[3]

oig = []
with jsonlines.open(os.path.join(DATA_DIR, data_path), "r") as f:
    for line in f.iter():
        oig.append(line)

len(oig)

210282

### ShareGPT Korean dataset

In [49]:
data_path = data_list[4]

with open(os.path.join(DATA_DIR, data_path), "r") as f:
    sharegpt = json.load(f)

len(sharegpt)

KeyboardInterrupt: 

### KULLM-v2 dataset

In [50]:
data_path = data_list[5]

kullm = []
with jsonlines.open(os.path.join(DATA_DIR, data_path), "r") as f:
    for line in f.iter():
        kullm.append(line)

len(kullm)

152630

In [52]:
# print(koalpaca_v1_0[0])
print(koalpaca_v1_1[0])
print(kochatgpt[0])
print(korquad[0])
print(oig[0])
# print(sharegpt[0])
print(kullm[-11])

{'instruction': '양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?', 'output': '양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.', 'url': 'https://kin.naver.com/qna/detail.naver?d1id=11&dirId=1116&docId=55320268'}
{'prompt': '불고기용 고기 한우에요?', 'completion': "'저는 인공지능 챗봇이며, 직접적으로 식품에 관한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 하지만 한우는 대표적인 고급 육류로 알려져 있기 때문에, 한우를 사용하는 경우도 많습니다. 알러지나 개별 건강 상태에 따라 다를 수 있으니 충분한 정보 수집 후에 선택해 주시기 바랍니다.", 'tokens': 193}
{'source': 'korquad-chat', 'text': '<sys>1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의

In [53]:
import re
from tqdm import tqdm

def to_dialog(data, inst_key, out_key):
    dialog = []
    for sample in tqdm(data, total=len(data)):
        output = re.sub(r".(\d).", r".\n\1.", sample[out_key])
        
        dialog.append(
            [{"from": "human", "value": sample[inst_key]},
            {"from": "bot", "value": output}]
        )
    
    return dialog
        

print("Unifying except for Korquad dataset")
unified_data = []

# koalpacav1.1
unified_data.extend(to_dialog(koalpaca_v1_1, "instruction", "output"))
# kochatgpt
unified_data.extend(to_dialog(kochatgpt, "prompt", "completion"))
# OIG
unified_data.extend(to_dialog(oig, "user_translated", "chip2_translated"))
# ShareGPT
# unified_data.extend(to_dialog(sharegpt, "instruction", "output"))
    
# KULLM-v2
processed_kullm = []
for sample in tqdm(kullm, total=len(kullm)):
    if sample["input"]:
        inst = sample["instruction"].strip()+"\n\n"+sample["input"].strip()
    else:
        inst = sample["instruction"].strip()
        
    if len(inst) <= 15:
        continue
    
    output = re.sub(r".(\d).", r".\n\1.", sample["output"])
        
    processed_kullm.append(
        [{"from": "human", "value": inst},
         {"from": "bot", "value": output}]
    )
print(len(processed_kullm))
unified_data.extend(processed_kullm)

# unified_data.extend(koalpaca_v1_0)
    
len(unified_data)

Unifying except for Korquad dataset


100%|██████████| 152630/152630 [00:29<00:00, 5132.82it/s]

137583


381020

### Processing Koquard dataset

In [54]:
for sample in korquad:#, total=len(korquad)):
    try:
        turns = sample["text"].split("\n")
        context = turns[0].split(">")[1].strip()
        dialog = []
        for turn in turns[1:]:
            splited = turn.split(">")
            role = splited[0][1:]
            text = splited[1].strip()
            dialog.append({"role": role, "text": text})
            
        assert dialog[0]["role"]=="usr" and len(dialog)%2==0
        
        inst_res = []
        for i in range(0, len(dialog), 2):
            user = dialog[i]["text"]
            if i==0:
                user = context.strip()+"\n\n"+user.strip()
            bot = dialog[i+1]["text"]
        
            inst_res.extend(
                [{"from": "human", "value": user},
                 {"from": "bot", "value": bot}]
            )
        
        unified_data.append(inst_res)
    except:
        pass
    
len(unified_data)

390248

In [22]:
import torch
import transformers
from transformers import (
    MODEL_FOR_CAUSAL_LM_MAPPING,
    Trainer,
    set_seed,
    AutoConfig,
    CONFIG_MAPPING
)

import sys
sys.path.append("../")

from model_utils.conversation import get_conv_template, SeparatorStyle
from typing import Optional, Dict, Sequence

IGNORE_INDEX = -100

def preprocess(
    sources,
    tokenizer: transformers.PreTrainedTokenizer,
) -> Dict:
    conv = get_conv_template("elementgpt_for_general")
    roles = {"human": conv.roles[0], "bot": conv.roles[1]}

    # Apply prompt templates
    conversations = []
    for i, source in enumerate(sources):
        if roles[source[0]["from"]] != conv.roles[0]:
            # Skip the first one if it is not from human
            source = source[1:]

        conv.messages = []
        for j, sentence in enumerate(source):
            role = roles[sentence["from"]]
            assert role == conv.roles[j % 2], f"{i}"
            conv.append_message(role, sentence["value"])
        conversations.append(conv.get_prompt())
        
    # Tokenize conversations
    input_ids = []
    targets = []
    for i, conversation in enumerate(tqdm(conversations, desc="Tokenizing inputs")):
        input_ids_sample = tokenizer(
            conversation,
            return_tensors="pt",
            padding="longest",
            max_length=tokenizer.model_max_length,
            truncation=True,
        ).input_ids[0]
        
        input_ids.append(input_ids_sample)
        targets.append(input_ids_sample.clone())

    assert conv.sep_style == SeparatorStyle.ADD_COLON_TWO

    print("Masking targets...")
    # Mask targets
    sep = conv.sep + conv.roles[1] + ": "
    for conversation, target in tqdm(zip(conversations, targets), total=len(conversations)):
        total_len = int(target.ne(tokenizer.pad_token_id).sum()) + 1

        rounds = conversation.split(conv.sep2)
        cur_len = 1
        target[:cur_len] = IGNORE_INDEX

        for i, rou in enumerate(rounds):
            if rou == "":
                break

            parts = rou.split(sep)
            if len(parts) != 2:
                break
            parts[0] += sep
            round_len = len(tokenizer(rou).input_ids)
            instruction_len = len(tokenizer(parts[0]).input_ids) - 2

            target[cur_len : cur_len + instruction_len] = IGNORE_INDEX

            cur_len += round_len
        target[cur_len:] = IGNORE_INDEX

        if False:
            z = target.clone()
            z = torch.where(z == IGNORE_TOKEN_ID, tokenizer.unk_token_id, z)
            rank0_print(tokenizer.decode(z))

        if cur_len < tokenizer.model_max_length:
            if cur_len != total_len:
                target[:] = IGNORE_INDEX
                print(
                    f"WARNING: tokenization mismatch: {cur_len} vs. {total_len}.",
                    f" (ignored)"
                )
                print(conversation)

    return dict(
        input_ids=input_ids,
        labels=targets,
    )

In [23]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    "EleutherAI/polyglot-ko-1.3b",
    cache_dir=None,
    model_max_length=1024,
    padding_side="right",
    use_fast=False,
)

processed = preprocess(unified_data[275282:275400], tokenizer)

Tokenizing inputs: 100%|██████████| 118/118 [00:00<00:00, 460.79it/s]


Masking targets...


 37%|███▋      | 44/118 [00:00<00:00, 438.70it/s]

호기심 많은 유저와 인공지능 어시스턴트 간의 채팅. 어시스턴트는 사용자의 질문에 도움이 되고 상세하며 정중한 답변을 합니다. USER:ASSISTANT: instruction
다음은 이 SRT에 대한 분석입니다.
1. 부터:.
0..
0..
3..
6. -.
0..
0..
0..
9.* 이.
1.은 발표자가 청중에게 인사하고 참석에 대한 감사를 표하는 것으로 시작됩니다.* 다음으로 발표자는 자신이 활동가는 아니지만 청중이 영감을 얻을 수 있기를 바라며 자신의 이야기를 나누고 싶다고 말합니다.* 이 부분은 연설의 시작점이며 청중이 발표자에 대한 관심을 불러일으키기 위한 것입니다.
1. 약속:.
0..
0..
1.7 -.
0..
1..
1..
9.* 이 섹션에서 발표자는 어릴 적 어머니의 노력에 영감을 받아 어릴 때부터 배우를 꿈꾸게 된 자신의 성장 경험을 공유하기 시작합니다.* 이 섹션은 전체 연설에서 이어지며 청중에게 발표자의 배경과 동기를 소개합니다.
1. 턴:.
0..
1..
1.4 -.
0..
1..
3..
6.* 이 섹션은 연설의 전환점입니다. 발표자는 어렸을 때부터 연기를 해왔다고 밝히며 자신이 출연했던 어린이 프로그램을 공유합니다.* 자신의 연기 경력에 대해 이야기하던 중 화자가 갑자기 감정이 격해진 듯 잠시 멈춘 후 계속합니다.* 이 부분은 발표자의 연기 경력에 감정적인 요소를 도입하여 전체 연설에 깊이와 의미를 더했습니다.
1. 합산:.
0..
1..
3..
6. -.
0..
2..
5..
0.* 이 섹션에서 발표자는 계속해서 자신의 성장 경험을 공유하고 연기에 대한 사랑과 경력에 대한 추구를 강조합니다.* 발표자는 청중의 경청과 지지에 감사를 표하고 자신의 이야기가 청중에게 영감을 주었으면 좋겠다는 바람을 표현하며 마무리합니다.* 이 섹션은 연설을 마무리하며 연설의 내용을 요약하고 다음과 같은 연설자의 열정과 목표를 강조하는 것을 목표로 합니다.<|endoftext|>
호기심 많은 유저와 인공지능 어시스턴트 간의 채팅. 어시스턴트는 사용자의 질

100%|██████████| 118/118 [00:00<00:00, 500.02it/s]


In [38]:
tokenizer.decode(processed["input_ids"][0])

'호기심 많은 유저와 인공지능 어시스턴트 간의 채팅. 어시스턴트는 사용자의 질문에 도움이 되고 상세하며 정중한 답변을 합니다. USER: 언제 그녀가 관계를 재건하고 싶어한다고 생각해야 하나요?포인트 ASSISTANT: 다음은 그녀가 관계를 재건할 의향이 있음을 나타내는 몇 가지 징후입니다.\n1. 귀하와 대화를 시작하거나 귀하가 연락을 취했을 때 긍정적으로 반응합니다.\n2. 그녀가 다시 농담이나 친근한 상호작용을 하기 시작합니다.\n3. 그녀가 당신과 함께 계획을 세우거나 함께 어울리자고 초대합니다.\n4. 그녀는 귀하의 행동에 대한 긍정적인 변화를 인정하고 이에 대해 감사를 표합니다.\n5. 과거에 상처를 준 행동에 대해 사과합니다.그러나 관계를 회복하려면 양쪽 모두의 시간과 노력이 필요하다는 점을 명심하세요. 공개적이고 정직하게 소통하고, 서로의 말에 귀를 기울이며, 발생하는 모든 문제를 기꺼이 해결하려는 자세가 중요합니다.<|endoftext|>'

In [55]:
import json

with open("../data/unified_instruction.json", "w", encoding="utf-8") as f:
    json.dump(unified_data, f, ensure_ascii=False) # ensure_ascii로 한글이 깨지지 않게 저장

In [56]:
len(unified_data)

390248

In [160]:
import sys
sys.path.append("../")
from model_utils.conversation import get_conv_template

conv = get_conv_template("elementgpt_for_general")
roles = {"human": conv.roles[0], "bot": conv.roles[1]}

conversations = []
for i, source in enumerate(unified_data[-10:]):
    if roles[source[0]["from"]] != conv.roles[0]:
        # Skip the first one if it is not from human
        source = source[1:]

    conv.messages = []
    for j, sentence in enumerate(source):
        role = roles[sentence["from"]]
        assert role == conv.roles[j % 2], f"{i}"
        conv.append_message(role, sentence["value"])
    conversations.append(conv.get_prompt())

In [162]:
conversations[2][]

'호기심 많은 유저와 인공지능 어시스턴트 간의 채팅. 어시스턴트는 사용자의 질문에 도움이 되고 상세하며 정중한 답변을 합니다. USER: 울산경찰청 지능범죄수사대는 2018년 3월 16일 자유한국당 소속 울산시장인 김기현의 측근 비리를 포착해 울산시청을 압수수색하고, 김기현 동생에 대한 체포영장을 발부받아 수사에 착수했다. 그에 대해 장제원 자유한국당 수석대변인은 2018년 3월 22일 논평을 내고 경찰을 겨냥해 "정권의 사냥개가 광견병까지 걸려 정권의 이익을 위해서라면 닥치는 대로 물어뜯기 시작했다. 미친개는 몽둥이가 약이다" 등의 원색적인 비난을 쏟아냈다. 이에 대해 여론이 싸늘해 지자 장제원은 2018년 3월 28일 자신의 페이스북에서 논란이 된 논평에 대해 \'경찰 전체가 아닌 울산경찰청장을 비롯한 일부 정치 경찰을 명시한 것\'이라고 해명하며 \'권력을 추종하는 정치경찰들을 반드시 추방해야 한다\'고 재차 주장했다.\n\n최근 언론에서 울산시장 김기현 측근 비리와 관련된 뉴스가 자주 보이는데, 그에 대한 대응책이 어떻게 될까요? ASSISTANT: 경찰이 울산시청을 압수수색하고, 김기현 시장의 측근에 대한 수사를 진행하고 있다고 합니다.<|endoftext|>USER: 그렇군요. 그런데 자유한국당 수석대변인이 경찰을 겨냥한 언급이 있다고 들었습니다. 어떻게 생각하십니까? ASSISTANT: 그에 대해 장제원 수석대변인은 경찰 전체가 아닌 몇몇 정치경찰의 문제점을 지적한 것이라고 해명하였습니다.<|endoftext|>USER: 장제원 씨는 정치경찰의 추방을 주장하고 있다는데, 이에 대해 어떻게 생각하십니까? ASSISTANT: 저는 권력을 추종하는 정치경찰이 있다면 이에 대한 대응책을 마련하고, 법을 지키며 공정한 수사를 해나가는 것이 바람직하다고 생각합니다.<|endoftext|>'